In [1]:
#Run with Python 3

import pandas as pd
import numpy as np
import urllib.request
import shutil
import re
import os
import pathlib
import argparse
import pdb
import threading
from bs4 import BeautifulSoup
import io

    
def soup_site(webdata):
    with io.open(webdata,'r',encoding='utf8') as f:
        info_data = f.read()
    '''opens site and turns it into a format to easily parse the DOM. Returns a Soup Object'''
    try:
        return BeautifulSoup(info_data, "lxml")
    except Exception as e:
        print(e)
        return -1

In [2]:
def get_web_data_from_link(webdata):
    return soup_site(webdata)

In [11]:
webdata = 'data.html'

def parse_studentInfo(studentInfo):
    studentID = ''
    studentName = ''
    
    # remove numbber if any
    if studentInfo.find('.') != -1:
        temp = studentInfo.split('.',1)
        stuInfo = temp[1]
    else:
        stuInfo = studentInfo
    
    # get id and name which are seperated by - 
    if studentInfo.find('-') != -1:
        temp = stuInfo.split('-')
        studentID = temp[0].strip()
        studentName = temp[1].strip()
    else:
        studentID = stuInfo.strip()
    return [studentID, studentName]


#group_problem ID 
group_problemID = [0]*6     # 6 problems, each problem starts from ID 0
# write to csv file
import csv
f = open("Assignment_group.csv", 'w', newline='', encoding="utf-8")
writer = csv.writer(f, delimiter=",")
writer.writerow(["ProblemID", "Student ID", "Student Name"])
#soup = get_web_data_from_link(webdata);
#print (soup)
count = 0
if soup != -1:
    postings = soup.findAll("div", {"class" : "posting fullpost"})
    #idx  = 2
    #print (postings[idx])
    #print ("TEXT TAG")
    #print (postings[idx].text)
    for posting in postings:
        writer.writerow('')
        count = count + 1
        problem_register = False
        pStrs = posting.find_all('p')
        gp = ''
        for pStr in pStrs:
            print (pStr)
            # trim string
            lineStr = pStr.text.strip()
            
            # skip the new line
            if len(lineStr) == 0:
                continue
            
            print (str(count) + "____"+ lineStr)
            
            # get problem and class info
            if lineStr. find("Problem") != -1:
                problem_register = True
                if lineStr.find ('(') != -1:
                    problemStr = lineStr.split('(')[0]
                else:
                    problemStr = lineStr

                # remove "problem" from sline and trim        
                problem = problemStr.replace("Problem",'')
                problem = problem.replace(":",'').strip()
                try:
                    problem = int(problem)
                    #print ("PROBLEM " + str(problem))
                    if problem <= 0 or problem > 6:
                        problemID = "Out of range"
                    else:
                        problemID = "{0:01d}_{1:02d}".format(problem, group_problemID[problem-1])
                        group_problemID[problem-1] = group_problemID[problem-1] + 1
                except ValueError:
                    problemID = "Wrong format"

                continue
            elif not problem_register:
                problemID = "NOT found"

                        
            # get student ID and name
            [studentID, studentName] = parse_studentInfo(lineStr)
            
            fproblemID = "{0:02d}_{1}".format(count, problemID)
            writer.writerow([fproblemID, studentID, studentName])
f.close()
print ("Summary: ")
print (group_problemID)


<p>Register your group (max of 3 students) and the problem by replying a post as following form:</p>
1____Register your group (max of 3 students) and the problem by replying a post as following form:
<p><b>Problem xxx (class L01)<br/></b></p>
1____Problem xxx (class L01)
<p><b>1. <b>1900001 - </b>Tran Thi A<br/></b></p>
1____1. 1900001 - Tran Thi A
<p><b>2. <b>1900002</b> - Nguyen Thi B<br/></b></p>
1____2. 1900002 - Nguyen Thi B
<p><b>3. <b>1900003 - </b>Le Thi C </b><br/></p>
1____3. 1900003 - Le Thi C
<p>Problem 2 (ClassL09)</p>
2____Problem 2 (ClassL09)
<p>1. 1920058 Lê Tất Thiện</p>
2____1. 1920058 Lê Tất Thiện
<p>2. 1927038 Trần Phương Tĩnh</p>
2____2. 1927038 Trần Phương Tĩnh
<p></p>
<p style="margin:0px 0px 10px;color:rgb(51,51,51);font-family:'Helvetica Neue', Helvetica, Arial, sans-serif;font-size:14px;font-style:normal;font-weight:400;letter-spacing:normal;text-align:left;text-indent:0px;text-transform:none;white-space:normal;word-spacing:0px;">Problem 5 (class L10)</p>
3___